# Notebook 1 - Basic concepts of Spark. 

This is a Scala notebook. To run any commands you first have to create a SparkContext.

If you connect to our cluster, then SparkContext is already loaded as variable sc

In [ ]:
sc

## Basic RDD Operations

**Create a RDD**


First create a collection of objects, for example a list of integers.

In [ ]:
val x = List(1,2,2,3,4)

Then you create an RDD from it. Here, we use "ParallelCollection" RDD, used by Spark to execute queries on it in parallel . 

In [ ]:
val rdd = sc.parallelize(x)

**RDD Transformations**

In [ ]:
val result = rdd.map(x => (x,x))

In [ ]:
result.collect().foreach(x => print(x + " "))

_Note that Scala is lazily evaluated, so we need to use "collect()" to actually produce the result._

In [ ]:
val result = rdd.flatMap(x => x.to(3))
result.collect().foreach(x => print(x + " "))

In [ ]:
rdd.filter(x => x!=2).collect().foreach(x => print(x + " "))

In [ ]:
rdd.map(x => x*x).flatMap(x => x.to(5)).filter(x => x!=3).collect().foreach(x => print(x + " "))

**RDD Actions (Reduce)**

Other actions, for example, are reduce() and map(). For more informations on supported RDD operations, check the documentation: https://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-operations. Please study the documentation, as it's expected that you are able to use these basic RDD actions to solve the exercises. 


In [ ]:
val sum = rdd.reduce(((x,y)=>x+y))

In [ ]:
val RDDNumbers= sc.parallelize((1).to(10))
val RDDsquared = RDDNumbers.map(x => x*x)

The RDD will be loaded to memory and the map transformation will be computed as soon as an action is called.

In [ ]:
RDDsquared.reduce((x,y) => x+y)

For key-value pairs you can use the action reduceByKey()

## Write your first MapReduce Job!

The standard example for MapReduce is WordCount. We use as input the diamonds dataset.

**1. Load the dataset as an RDD to SparkContext (sc)**

In [ ]:
val diamonds = sc.textFile("/home/adbs22/shared/diamonds.csv")

We use 'take' here to reduce the input to the first 20 elements:

In [ ]:
diamonds.collect().take(20).foreach(x => println(x))

In [ ]:
val entries = diamonds.flatMap(line => line.split(","))
entries.take(30).foreach(x => println(x))


In [ ]:
entries.collect().take(20).foreach(x => println(x))

**2. Create the MapReduce WordCount procedure**

In the map procedure key-value pairs are created. Spark provides the reducebyKey method to easily aggregate over the key-value pairs.

In [ ]:
val wordcounts = entries.map (word => (word, 1)).
reduceByKey( (x,y)  => x+y)


In [ ]:
var r = sc.parallelize(List(1,2,3,4,5),2)

In [ ]:
var r2 = r.takeSample(true,7,2)

The entire code for the Mapreduce wordcount procedure in Scala: 

In [ ]:
val ex1= sc.parallelize (List(1,2,3,4,5,6), 2)  // 2nd arg: no. partitions
val ex2= sc.parallelize (Array ("Alice", "Bob", "Caroline"))
val ex3= sc.parallelize (1 to 100)
println(ex3.getNumPartitions)   


### For more information see: https://spark.apache.org/docs/latest/rdd-programming-guide.html

# More basic RDD functions: 

In [ ]:
import util.Random.nextInt
val randomList = sc.parallelize(List.fill(40)(util.Random.nextInt))   

####  Compute the square root of each element in the list 'randomList', and then produce the sum of all square roots which are larger than 3:

In [ ]:
import scala.math.sqrt  // enables you to use the function sqrt() for computing the square root of a number
randomList.map(x => sqrt(x)).filter(x => x > 3).reduce(((x,y)=>x+y))

####  Produce the minimal value of 'randomList'.

In [ ]:
val minimal = randomList.reduce( (x,y) => x.min(y) )

####  Produce for the 'randomList' the average absolute difference to the minimal value. 

In [ ]:
val averageDifference = randomList.map(x => (x - minimal).abs  ).reduce(_+_) / randomList.count()

####  You are given a random list of words

In [ ]:
val words = sc.parallelize(List("key", "data", "car", "fish"))

####  Consider the following two predefined functions on String.

In [ ]:
def getTails (word:String) =  word.tails.toList.filter(x => x.length > 0)
getTails("Example")

In [ ]:
def getInitials (word:String) = word.inits.toList.filter(x => x.length > 0)
getInitials("Example")

####  Using the functions above, produce a single list containing all substrings of elements from the list 'words'. A formal definition of 'substring', is given here: https://en.wikipedia.org/wiki/Substring#Substring.

In [ ]:
def allSubstrings = words.flatMap(getInitials).flatMap(getTails)
allSubstrings.collect()